In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 3.5 MB/s eta 0:00:00


In [ ]:
pip install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import openai
import pinecone
from sentence_transformers import SentenceTransformer

In [ ]:
# Setting OpenAI API key
openai.api_key = "OpenAI API key"

# Setting Pinecone API key
pinecone_api_key = "Pinecone API key"
pinecone_index_name = "qa_bot_index"

# Connecting to Pinecone
#pinecone = pinecone.init(api_key="YOUR_PINECONE_API_KEY")
pinecone_client = pinecone.Pinecone(api_key=pinecone_api_key)

In [ ]:
# Function to create or get Pinecone index
def create_or_get_index():
    if pinecone_index_name not in pinecone.list_indexes():
        pinecone.create_index(index_name=pinecone_index_name)
    return pinecone_index_name


In [ ]:

# Function to insert vectors into Pinecone
def insert_vectors(vectors):
    index_name = create_or_get_index()
    pinecone.upsert_items(index_name=index_name, items=vectors)


In [ ]:

# Function to search Pinecone index
def search_pinecone(query_vector, num_results=1):
    index_name = create_or_get_index()
    results = pinecone.query(index_name=index_name, query_vector=query_vector, top_k=num_results)
    return results

In [ ]:

# Function to generate answer using OpenAI API
def generate_answer(prompt):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=0.5,
        max_tokens=150,
        n=1
    )
    return response.choices[0].text.strip()


In [ ]:

# Loading a pre-trained sentence embedding model
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Creating Convert to Vector function

def convert_to_vector(text):
    # Generate an embedding for the given text
    vector = embedding_model.encode(text)
    return vector


In [ ]:

# Main function
def main():
    # Sample question
    question = "What is the mission of Yardstick ?"


    passage = generate_answer(question)


    passage_vector = convert_to_vector(passage)


    insert_vectors([(passage_vector, question)])

    new_question = "What range of services does Yardstick offer?"
    new_question_vector = convert_to_vector(generate_answer(new_question))
    results = search_pinecone(new_question_vector)


    if results:
        answer = results[0]["data"]
        print(f"Answer to '{new_question}': {answer}")
    else:
        print("No relevant answer found.")



In [ ]:
# Runing the main function
main()

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.